In [1]:
import robot
import map
import utils

import numpy as np
import gtsam
import matplotlib.pyplot as plt
import random

%matplotlib qt

# unitTest 1: Move robot and check odometrey measurements

In [2]:
car = robot.robot()

#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)

time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()

#set graphics
fig , ax = utils.setWorldMap()
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)
ax.legend(["ground truth","dead reckoning"])

#run and plot simulation
xcurrent_DR = hist_GT.copy()
while time <= timeFinal:
    meas_odom = car.moveAndMeasureOdometrey((dr,dtheta))
    
    #dead reckoning integration
    xcurrent_DR[2] += meas_odom.dtheta #update angle
    xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
    xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
    
    #log history
    hist_DR = np.vstack([hist_DR,xcurrent_DR])
    hist_GT = np.vstack([hist_GT,car.pose])

    #increase time
    time += dt

    #plot
    graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
    graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
    plt.draw()
    plt.pause(dt)

# unitTest 2: World Landmarks

In [8]:
np.random.seed(seed=2)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)
sigMin = -0.5
sigMax = 0.5

landmarks = [None] * N
for ii in range(N):
    sigs = np.random.uniform(low=sigMin, high=sigMax, size=(2,2))
    cov = sigs@sigs.T #enforce symmetric and positive definite: https://mathworld.wolfram.com/PositiveDefiniteMatrix.html
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 cov = cov)

priorMap = map.map(landmarks)
priorMap.plot(plotIndex = True,plotCov = True)

# unitTest 3: Move car and check landmark measurements

In [2]:
#------Build worldmap
fig , ax = utils.setWorldMap()
np.random.seed(seed=0)

N = 10 #number of landmarks
semantics = ("table","MEP","chair","pillar","clutter")
xrange = (-2,2)
yrange = (-1,3)

landmarks = [None] * N
for ii in range(N):
    landmarks[ii] = map.landmark(x = (np.random.rand()-0.5) * np.diff(xrange) + np.mean(xrange),
                                 y = (np.random.rand()-0.5) * np.diff(yrange) + np.mean(yrange),
                                 classLabel = np.random.choice(semantics),
                                 )
worldMap = map.map(landmarks)
worldMap.plot(ax = ax, plotIndex = True,plotCov = False)

#------Spawn Robot
car = robot.robot()
#control inputs
dr = 0.4 #[m]
dtheta = np.radians(20)
odom = (dr,dtheta)

#------Simulation parameters
time = 0 #[s]
dt = 0.5 
timeFinal = 10

#init history loggers
hist_GT = np.array(car.pose)
hist_DR = hist_GT.copy()
hist_LM = []

#set graphics
graphic_GT_traj, = plt.plot([], [],'ko-',markersize = 5)
graphic_DR_traj, = plt.plot([], [],'ro-',markersize = 5)

#run and plot simulation
xcurrent_DR = hist_GT.copy()
while time <= timeFinal:
    meas_odom = car.moveAndMeasureOdometrey((dr,dtheta))

    meas_lm = car.measureLandmarks(worldMap)

    #dead reckoning integration
    xcurrent_DR[2] += meas_odom.dtheta #update angle
    xcurrent_DR[0] += meas_odom.dr * np.cos(xcurrent_DR[2])
    xcurrent_DR[1] += meas_odom.dr * np.sin(xcurrent_DR[2])
    
    #log history
    hist_GT = np.vstack([hist_GT,car.pose])
    hist_DR = np.vstack([hist_DR,xcurrent_DR])
    hist_LM.append(meas_lm)

    #increase time
    time += dt

    #plot
    graphic_GT_traj.set_data(hist_GT[:,0],hist_GT[:,1])
    graphic_DR_traj.set_data(hist_DR[:,0],hist_DR[:,1])
    plt.draw()
    plt.pause(dt)

NameError: name 'sigMin' is not defined

In [6]:
hist_LM

[[],
 [],
 [[1.0701594120895108, -0.20295693106980556, 'clutter', 4]],
 [[0.36536309674958245, -0.09733431111698528, 'pillar', 5]],
 [[0.33089461050639296, -2.470397454531601, 'MEP', 1],
  [0.5876808646401244, -1.0058183858368215, 'pillar', 5]],
 [[0.49181304155054434, -3.1230007940713844, 'chair', 0],
  [0.8993851129131344, -1.865928148787276, 'MEP', 1],
  [0.9160159379531105, -0.9827685310844555, 'pillar', 5]],
 [[0.2908421184647328, -2.1505909583930687, 'chair', 0],
  [1.075850553754783, -1.4709132717962412, 'MEP', 1],
  [0.8356567069957432, -2.4798922130369676, 'MEP', 2],
  [0.7100949996643591, 0.36730931626650826, 'MEP', 9]],
 [[0.40112822861516795, -1.1471481284451788, 'chair', 0],
  [0.6246505515563463, -1.9494868475510958, 'MEP', 2]],
 [[0.6319737505208072, -0.5803367436685772, 'chair', 0],
  [0.6403603688312429, -1.4953398417943724, 'MEP', 2],
  [0.7972255812874691, 0.5956145770660458, 'clutter', 6]],
 [[0.6589328374272743, -0.8804567566677903, 'MEP', 2],
  [0.6157443067256247